### 날짜 사이 모든 날짜 리스트로 구해서 mongoDB에 저장

In [ ]:
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus
from xml.etree import ElementTree

from pyspark.conf import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType
from pyspark.sql.functions import array_contains, udf

from datetime import datetime, timedelta
from pymongo import MongoClient
import pandas as pd
import json
## processing_load.py
spark = SparkSession \
    .builder \
    .appName("multi") \
    .config("spark.mongodb.input.uri", "mongodb://localhost:27017") \
    .config("spark.mongodb.input.database","datalake") \
    .config("spark.mongodb.input.collection", "i") \
    .config("packages org.mongodb.spark:mongo-spark-connector_2.12:3.0.1") \
    .getOrCreate()
sc =spark.sparkContext

# pymongo connect
client = MongoClient('localhost',27017) # mongodb 27017 port
db = client.datalake

skey = "5752524157776a643837426953524b"
today = datetime.today() - timedelta(4)
today_4 = today.strftime("%Y%m%d")

url = "http://openapi.seoul.go.kr:8088/{0}/json/CardSubwayStatsNew/1/1000/".format(skey)

today = datetime.today() - timedelta(5)
today_5 = today.strftime("%Y%m%d")
start = '20190101'

def date_range(start, end):
    start = datetime.strptime(start, "%Y%m%d")
    end = datetime.strptime(end, "%Y%m%d")
    dates = [date.strftime("%Y%m%d") for date in pd.date_range(start, periods=(end-start).days+1)]
    return dates
    
dates = date_range(start, today_5)

## openAPI에서 받아온 데이터를 전부 mongoDB에 저장
for i in date:
    responseBody = urlopen(url+i).read().decode('utf-8')
    jsonArray = json.loads(responseBody)
    storeInfosArray= jsonArray["CardSubwayStatsNew"]["row"]
    if storeInfosArray is not None:
        for j in range(len(storeInfosArray)):
            db.i.insert_one(storeInfosArray[j])
            

In [8]:
db.i.remove({'USE_DT':'20210814'})

<ipython-input-8-d8f9cccf4e54>:1: DeprecationWarning: remove is deprecated. Use delete_one or delete_many instead.
  db.i.remove({'USE_DT':'20210814'})


{'n': 3606, 'ok': 1.0}

In [9]:
## 확인용

print('MongoDB Database : ',client.list_database_names())

print('MongoDB Collection : ',db.list_collection_names(),'\n')

# 검색
print('Data')
result = db.i.find({"USE_DT":"20210814"})
print(result.count())
for i in result[:10]:
    print(i)

MongoDB Database :  ['admin', 'config', 'datalake', 'local', 'test']
MongoDB Collection :  ['i'] 

Data


<ipython-input-9-d97d3d6adf86>:10: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  print(result.count())


601
{'_id': ObjectId('611c61ecaaab34c81dbab098'), 'USE_DT': '20210814', 'LINE_NUM': '1호선', 'SUB_STA_NM': '서울역', 'RIDE_PASGR_NUM': 21222.0, 'ALIGHT_PASGR_NUM': 23912.0, 'WORK_DT': '20210817'}
{'_id': ObjectId('611c61ecaaab34c81dbab099'), 'USE_DT': '20210814', 'LINE_NUM': '1호선', 'SUB_STA_NM': '동묘앞', 'RIDE_PASGR_NUM': 9368.0, 'ALIGHT_PASGR_NUM': 9560.0, 'WORK_DT': '20210817'}
{'_id': ObjectId('611c61ecaaab34c81dbab09a'), 'USE_DT': '20210814', 'LINE_NUM': '1호선', 'SUB_STA_NM': '시청', 'RIDE_PASGR_NUM': 5601.0, 'ALIGHT_PASGR_NUM': 3452.0, 'WORK_DT': '20210817'}
{'_id': ObjectId('611c61ecaaab34c81dbab09b'), 'USE_DT': '20210814', 'LINE_NUM': '1호선', 'SUB_STA_NM': '종각', 'RIDE_PASGR_NUM': 13776.0, 'ALIGHT_PASGR_NUM': 12412.0, 'WORK_DT': '20210817'}
{'_id': ObjectId('611c61ecaaab34c81dbab09c'), 'USE_DT': '20210814', 'LINE_NUM': '1호선', 'SUB_STA_NM': '종로3가', 'RIDE_PASGR_NUM': 18102.0, 'ALIGHT_PASGR_NUM': 17077.0, 'WORK_DT': '20210817'}
{'_id': ObjectId('611c61ecaaab34c81dbab09d'), 'USE_DT': '20210814'

In [19]:
from pymongo import MongoClient
from datetime import datetime
import pandas as pd

# pymongo connect
client = MongoClient('localhost',27017) # mongodb 27017 port
db = client.ojo_db
db.test

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'ojo_db'), 'test')

In [25]:
day = datetime.now() - pd.DateOffset(days=2)

In [26]:
nowtime = day.strftime('%Y-%m-%d')
nowtime

'2021-09-22'

In [29]:
a = db.test.find({'date_st': nowtime})

In [30]:
for i in a:
    print(i)

{'_id': ObjectId('614c382caaa02972f5452a2b'), 'area': '제주', '확진자수': 2803, 'date_st': '2021-09-22', 'Stage': '3 단계', 'Description': '- 제주 전지역 4단계 (21.9.23~10.3.)'}
{'_id': ObjectId('614c382caaa02972f5452a2c'), 'area': '경남', '확진자수': 10983, 'date_st': '2021-09-22', 'Stage': '3 단계', 'Description': '- 경남 전지역 3단계 (21.9.6~10.3.)'}
{'_id': ObjectId('614c382caaa02972f5452a2d'), 'area': '경북', '확진자수': 7848, 'date_st': '2021-09-22', 'Stage': '3 단계', 'Description': '- 경북 일부지역 3단계 (21.9.6~10.3.)\n- 문경시, 상주시 2단계 (21.9.6~10.3.)\n- 군위군, 의성군, 청송군, 영양군, 영덕군, 청도군, 고령군, 성주군, 예천군, 봉화군, 울릉군, 울진군 1단계 (21.9.6~10.3.)'}
{'_id': ObjectId('614c382caaa02972f5452a2e'), 'area': '전남', '확진자수': 2933, 'date_st': '2021-09-22', 'Stage': '3 단계', 'Description': '- 전남 전지역 3단계 (21.9.6~10.3.)'}
{'_id': ObjectId('614c382caaa02972f5452a2f'), 'area': '전북', '확진자수': 4179, 'date_st': '2021-09-22', 'Stage': '3 단계', 'Description': '- 전북 일부지역 3단계 (21.9.6~10.3.)\n- 정읍시, 남원시, 김제시, 완주군(혁신도시 갈산리 제외), 진안군, 무주군, 장수군, 임실군, 순창군, 고창군, 부안군 2단계 (2

In [6]:
from pymongo import MongoClient
from datetime import datetime
import pandas as pd

# pymongo connect
client = MongoClient('localhost',27017) # mongodb 27017 port
db = client.ojo_db
db.test

day = (datetime.now() - pd.DateOffset(days=2)).strftime('%Y-%m-%d')

# nowtime = day
# nowtime

a = db.test.find({'date_st': nowtime})

for i in a:
    print(type(i))
    for j in i.items():
        print(j)

<class 'dict'>
('_id', ObjectId('614c382caaa02972f5452a2b'))
('area', '제주')
('확진자수', 2803)
('date_st', '2021-09-22')
('Stage', '3 단계')
('Description', '- 제주 전지역 4단계 (21.9.23~10.3.)')
<class 'dict'>
('_id', ObjectId('614c382caaa02972f5452a2c'))
('area', '경남')
('확진자수', 10983)
('date_st', '2021-09-22')
('Stage', '3 단계')
('Description', '- 경남 전지역 3단계 (21.9.6~10.3.)')
<class 'dict'>
('_id', ObjectId('614c382caaa02972f5452a2d'))
('area', '경북')
('확진자수', 7848)
('date_st', '2021-09-22')
('Stage', '3 단계')
('Description', '- 경북 일부지역 3단계 (21.9.6~10.3.)\n- 문경시, 상주시 2단계 (21.9.6~10.3.)\n- 군위군, 의성군, 청송군, 영양군, 영덕군, 청도군, 고령군, 성주군, 예천군, 봉화군, 울릉군, 울진군 1단계 (21.9.6~10.3.)')
<class 'dict'>
('_id', ObjectId('614c382caaa02972f5452a2e'))
('area', '전남')
('확진자수', 2933)
('date_st', '2021-09-22')
('Stage', '3 단계')
('Description', '- 전남 전지역 3단계 (21.9.6~10.3.)')
<class 'dict'>
('_id', ObjectId('614c382caaa02972f5452a2f'))
('area', '전북')
('확진자수', 4179)
('date_st', '2021-09-22')
('Stage', '3 단계')
('Description', '- 전북 